In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sy
np.set_printoptions(suppress=True, precision=4)
from scipy.linalg import expm, logm
import scipy
from numba import jit, njit
from functools import partial

from plot_helpers import plotCoordinateFrame, set_axes_equal

In [2]:
@njit
def skew(u):
    return np.array([[   0, -u[2],  u[1]],
                    [ u[2],     0, -u[0]],
                    [-u[1],  u[0],     0]])

@njit
def K_from_vec(x):
    K = np.zeros((3,4))
    K[0,0] = x[0]
    K[1,1] = x[1]
    K[0,1] = x[2]
    K[2,2] = 1
    K[0:2,2] = x[3:5]
    return K

@njit
def vec_from_K(K):
    return np.array([K[0,0], K[1,1], K[0,1], K[0,2], K[1,2]])

@njit
def SE3_from_vec(u):
    # https://www.ethaneade.com/lie.pdf
    w = u[:3]
    p = u[3:]
    wx = skew(w)
    theta = np.linalg.norm(w)
    if np.abs(theta) < 0.0001:
        R = np.eye(3) + wx + wx@wx/2 + wx@wx@wx/6
        V = np.eye(3) + wx/2 + wx@wx/6 + wx@wx@wx/24
    
    else:
        A = np.sin(theta) / theta
        B = (1 - np.cos(theta)) / theta**2
        C = (1 - A) / theta**2

        R = np.eye(3) + A*wx + B*wx@wx
        V = np.eye(3) + B*wx + C*wx@wx

    T = np.hstack((R, (V@p).reshape((3,1))))
    T = np.vstack(( T, np.array([[0.0,0,0,1]]) ))
    return T
 
@njit
def vec_from_SE3(T):
    # https://www.ethaneade.com/lie.pdf
    xi = np.zeros(6)
    
    # Log on the rotations
    R = T[:3,:3]
    p = T[:3,3].copy()
    theta = np.arccos( (np.trace(R) - 1) / 2 )
    x = (R - R.T)*theta / (2*np.sin(theta))
    
    xi[0] = R[2,1] - R[1,2]
    xi[1] = R[0,2] - R[2,0]
    xi[2] = R[1,0] - R[0,1]
    
    if theta != 0:
        xi[:3] *= theta / (2*np.sin(theta))
        
    # And on the translation
    wx = skew(xi[:3])
    if theta < .0001:
        V = np.eye(3) + wx/2 + wx@wx/6 + wx@wx@wx/24
    else:
        A = np.sin(theta) / theta
        B = (1 - np.cos(theta)) / theta**2
        C = (1 - A) / theta**2
        V = np.eye(3) + B*wx + C*wx@wx
        
    Vinv = np.linalg.inv(V)
    xi[3:] = Vinv@p
    return xi

@njit
def to_homogen(p):
    return np.hstack(( p, np.ones((p.shape[0], 1)) ))

@njit
def from_homogen(p):
    p /= p[:,-1:]
    return p[:,:-1]

In [3]:
def nder(f, x, eps=1e-4):
    fx = f(x)
    N = x.shape[0]
    M = fx.shape[0]
    d = np.zeros((M,N))
    for i in range(N):
        temp = x.copy()
        temp[i] += eps
        d[:,i] = (f(temp) - fx) / eps
        
    return d

## Test seperate derivatives

In [4]:
g = lambda p: np.array([p[0]/p[2], p[1]/p[2]])
g_p = lambda p: np.array([[1/p[2], 0, -p[0]/p[2]**2],
                         [0, 1/p[2], -p[1]/p[2]**2]])


p = np.array([3,2,1.])
a = g_p(p)
n = nder(g, p, 1e-7)
print("dg / dp")
print("Analytical\n", a)
print("Numerical\n", n)
assert(np.allclose(a,n))

dg / dp
Analytical
 [[ 1.  0. -3.]
 [ 0.  1. -2.]]
Numerical
 [[ 1.  0. -3.]
 [ 0.  1. -2.]]


In [17]:
@njit
def f(Kvec, Tvec, P, Tdelta=None):
    if Tdelta is None:
        return K_from_vec(Kvec)@SE3_from_vec(Tvec)@np.append(P, 1)
    else:
        return K_from_vec(Kvec)@SE3_from_vec(Tvec)@SE3_from_vec(Tdelta)@np.append(P, 1)


f_P = lambda Kvec, Tvec, P : K_from_vec(Kvec)[:,:3]@SE3_from_vec(Tvec)[:3,:3]

Kvec = np.array([1520., 1500.,  0.3, 300.,  250.])
Tvec = np.array([.2, -10*np.pi/180, .1, 0.7, 0.3, 0.1])
P = np.array([3, 2, 1.])

a = f_P(Kvec, Tvec, P)
n = nder(lambda Pp : f(Kvec, Tvec, Pp), P, 1e-7)
print("df / dP")
print("Analytical\n", a)
print("Numerical\n", n)
assert(np.allclose(a,n))

df / dP
Analytical
 [[1544.1196 -119.4307   42.7913]
 [ 167.5254 1509.9159  -67.7443]
 [   0.1821    0.1887    0.965 ]]
Numerical
 [[1544.1196 -119.4307   42.7913]
 [ 167.5254 1509.9159  -67.7443]
 [   0.1821    0.1887    0.965 ]]


In [12]:
def f_K(Kvec, Tvec, P):
    Pp = SE3_from_vec(Tvec)@np.append(P,1)
    return np.array([[Pp[0],     0, Pp[1], Pp[2],     0],
                     [    0, Pp[1],     0,     0, Pp[2]],
                     [    0,     0,     0,     0,     0]])

a = f_K(Kvec, Tvec, P)
n = nder(lambda Kk : f(Kk, Tvec, P), Kvec, 1e-7)
print("df / dK")
print("Analytical\n", a)
print("Numerical\n", n)
assert(np.allclose(a,n))

df / dK
Analytical
 [[3.2158 0.     2.3063 2.0795 0.    ]
 [0.     2.3063 0.     0.     2.0795]
 [0.     0.     0.     0.     0.    ]]
Numerical
 [[3.2158 0.     2.3063 2.0795 0.    ]
 [0.     2.3063 0.     0.     2.0795]
 [0.     0.     0.     0.     0.    ]]


In [13]:
def f_T(Kvec, Tvec, P):
    R = SE3_from_vec(Tvec)[:3,:3]
    mat = np.hstack(( -R@skew(P), R ))
    # mat = np.vstack(( mat, np.zeros((1,6)) ))
    return K_from_vec(Kvec)[:3,:3]@mat

a = f_T(Kvec, Tvec, P)
n = nder(lambda Tdelta : f(Kvec, Tvec, P, Tdelta), np.zeros(6), 1e-7)
print("df / dT")
print("Analytical\n", a)
print("Numerical\n", n)
assert(np.allclose(a,n))

df / dT
Analytical
 [[  205.0133  1415.7458 -3446.5314  1544.1196  -119.4307    42.7913]
 [-1645.4045   370.7584  4194.6968   167.5254  1509.9159   -67.7443]
 [    1.7413    -2.7129     0.2017     0.1821     0.1887     0.965 ]]
Numerical
 [[  205.0133  1415.7456 -3446.5316  1544.1196  -119.4307    42.7913]
 [-1645.4047   370.7583  4194.6967   167.5254  1509.9159   -67.7443]
 [    1.7413    -2.7129     0.2017     0.1821     0.1887     0.965 ]]


## Full Jacobians together

In [14]:
a = g_p(f(Kvec, Tvec, P))@f_P(Kvec, Tvec, P)
n = nder(lambda Pp : g(f(Kvec, Tvec, Pp)), P, 1e-7)
print("dg / dP")
print("Analytical\n", a)
print("Numerical\n", n)
assert(np.allclose(a,n))

dg / dP
Analytical
 [[  510.3649  -297.9326 -1209.5879]
 [  -87.0456   552.4868  -920.6054]]
Numerical
 [[  510.3649  -297.9326 -1209.5878]
 [  -87.0456   552.4868  -920.6053]]


In [15]:
a = g_p(f(Kvec, Tvec, P))@f_K(Kvec, Tvec, P)
n = nder(lambda Kk : g(f(Kk, Tvec, P)), Kvec, 1e-7)
print("dg / dK")
print("Analytical\n", a)
print("Numerical\n", n)
assert(np.allclose(a,n))

dg / dK
Analytical
 [[1.5464 0.     1.1091 1.     0.    ]
 [0.     1.1091 0.     0.     1.    ]]
Numerical
 [[1.5464 0.     1.1091 1.     0.    ]
 [0.     1.1091 0.     0.     1.    ]]


In [16]:
a = g_p(f(Kvec, Tvec, P))@f_T(Kvec, Tvec, P)
n = nder(lambda Tdelta : g(f(Kvec, Tvec, P, Tdelta)), np.zeros(6), 1e-7)
print("dg / dK")
print("Analytical\n", a)
print("Numerical\n", n)
assert(np.allclose(a,n))

dg / dK
Analytical
 [[-2121.2432  4139.1285 -1914.5274   510.3649  -297.9326 -1209.5879]
 [-2393.6976  2674.7705  1831.5517   -87.0456   552.4868  -920.6054]]
Numerical
 [[-2121.243   4139.1291 -1914.5274   510.3649  -297.9326 -1209.5878]
 [-2393.6974  2674.7709  1831.5516   -87.0456   552.4868  -920.6053]]


In [18]:
x = np.arange((10)).reshape((5,2))

In [19]:
x

array([[0, 1],
       [2, 3],
       [4, 5],
       [6, 7],
       [8, 9]])

In [20]:
x.flatten()

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])